## Classification based on global embeddings extracted previously using pretrained ResNest101 and Inceptionv3

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim 
from torchvision import datasets
from torch.autograd import Variable
import numpy as np
import pandas as pd
import cv2
import argparse
import os
from PIL import Image, ImageEnhance, ImageOps
from tqdm import tqdm
import random

In [ ]:
load_dir = '../Global_embeddings'
batch_size = 32
epochs = 150
lr = 0.01
momentum = 0.9
weight_decay = 3e-3
grad_clip = 5.
seed = 0
use_cuda = False
experiment='../experiment'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.manual_seed(seed)

In [ ]:
# Features and labels
features_train = torch.load(os.path.join(load_dir, "birds_features_train.pt"), map_location=torch.device(device))
labels_train = torch.load(os.path.join(load_dir, "birds_labels_train.pt"), map_location=torch.device(device))

features_val = torch.load(os.path.join(load_dir, "birds_features_val.pt"), map_location=torch.device(device))
labels_val = torch.load(os.path.join(load_dir, "birds_labels_val.pt"), map_location=torch.device(device))

# Dataloaders
features_tensor = torch.stack([i for i in features_train])
labels_tensor = torch.stack([i for i in labels_train])
train_data = torch.utils.data.TensorDataset(features_tensor, labels_tensor) 

features_tensor = torch.stack([torch.Tensor(i) for i in features_val])
labels_tensor = torch.stack([i for i in labels_val])
val_data = torch.utils.data.TensorDataset(features_tensor,labels_tensor)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=0)

In [ ]:
# Model
class Classifier(nn.Module):
    def __init__(self,embedding_dim):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(embedding_dim, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 20)

    def forward(self, x):
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Classifier(features_train[0].shape[0])

if use_cuda:
    print('Using GPU')
    model.cuda()
else:
    print('Using CPU')

# Optimizer, LR, and criterion
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

#optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
criterion = torch.nn.CrossEntropyLoss(reduction="mean")

# Training functions
def train_classifier(model, train_loader, optimizer, lr_scheduler, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = Variable(data.cuda()), Variable(target.cuda().long())
                
        else:
              data, target = Variable(data), Variable(target.long())
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        optimizer.step()
        lr_scheduler.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))

def validation_classifier(model, criterion, val_loader):
    model.eval()
    validation_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            if use_cuda:
                data, target = Variable(data.cuda()), Variable(target.cuda().long())
            else:
                data, target = Variable(data), Variable(target.long())
            output = model(data)
                    
            # sum up batch loss
            validation_loss += criterion(output, target).data.item()
            
            # get the index of the max log-probability
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    validation_loss /= len(val_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        validation_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))
    return(100. * correct / len(val_loader.dataset))

In [ ]:
# Training the classifier 
for epoch in range(1, epochs + 1):
    train_classifier(model, train_loader, optimizer, lr_scheduler, criterion, epoch)
    val_acc=validation_classifier(model, criterion, val_loader)
    if val_acc>=93:
        model_file = experiment + '/model_Inceptionv3_' + str(epoch) + '.pth'
        torch.save(model.state_dict(), model_file)
        print('Saved model to ' + model_file + '. You can run `python evaluate.py --model ' + model_file + '` to generate the Kaggle formatted csv file\n')